In [19]:
from PyPDF2 import PdfFileReader

In [20]:
def get_info(path):
    with open(path, 'rb') as f:
        pdf = PdfFileReader(f)
        info = pdf.getDocumentInfo()
        number_of_pages = pdf.getNumPages()
    print(info)
    print(number_of_pages)
    author = info.author
    creator = info.creator
    producer = info.producer
    subject = info.subject
    title = info.title
if __name__ == '__main__':
    path = 'SE_Project_Report_Template.pdf'
    get_info(path)

{'/Author': 'John', '/CreationDate': "D:20140112140759-06'00'", '/Creator': 'PScript5.dll Version 5.2.2', '/ModDate': "D:20140112140759-06'00'", '/Producer': 'Acrobat Distiller 9.5.5 (Windows)', '/Title': 'Microsoft Word - SE_Project_Report_Template.docx'}
78


In [73]:
def text_extractor(path):
    pdf_text = ""
    with open(path, 'rb') as f:
        pdf = PdfFileReader(f)
        number_of_pages = pdf.getNumPages()
        for i in range(11, number_of_pages):
            # get the 11th page
            page = pdf.getPage(i)
            text = page.extractText()
            pdf_text = pdf_text + text.replace("\n", " ")
            #pdf_text.append(text.replace("\n", " "))    
        return pdf_text
if __name__ == '__main__':
    path = 'SE_Project_Report_Template.pdf'
text = text_extractor(path)

## Text Cleaning

In [74]:
text = text.lower()
import itertools
import re

text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
words = text.split()

len(words)

15883

In [75]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [76]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [77]:
words = [w for w in words if not w in stop_words]

In [79]:
text = ' '.join(words)

In [81]:
remove_digits = str.maketrans('', '', digits)
text = text.translate(remove_digits)

In [83]:
words = text.split()
text = ' '.join(words)

In [84]:
from nltk import sent_tokenize
sentences = sent_tokenize(text)
len(sentences)

884

In [85]:
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in words]
print(stripped[:100])

['projectdescription', 'projectoverviewa', 'brief', 'description', 'product', 'produced', 'getting', 'details', 'thepurposeoftheprojecta', 'theuserbusinessorbackground', 'oftheprojecteffortcontent', 'content', 'motivation', 'examples', 'considerations', 'short', 'description', 'business', 'done', 'context', 'situation', 'triggered', 'development', 'effort', 'also', 'describe', 'work', 'user', 'intends', 'delivered', 'product', 'motivation', 'without', 'statement', 'project', 'lacks', 'justification', 'direction', 'considerations', 'consider', 'whether', 'user', 'probl', 'em', 'serious', 'whether', 'needs', 'solved', 'b', 'goalsoftheproject', 'note', 'item', 'following', 'one', 'together', 'cover', '', 'objectives', 'success', 'criteria', 'project', 'item', 'specified', 'bruegge', '', 'dutoit', '', 'content', 'boils', 'one', 'sentence', 'mo', 'st', 'sentences', 'say', 'want', 'product', 'state', 'real', 'reason', 'product', 'developed', 'motivation', 'danger', 'purpose', 'may', 'get', '

In [86]:
len(stripped)

9012

In [87]:
from string import digits
text = ' '.join(stripped)

In [88]:
words = text.split()
text = ' '.join(words)

## Summarizer

In [89]:
word2count = {}

In [90]:
from nltk import word_tokenize

In [91]:
for word in word_tokenize(text):
    if word not in stop_words:
        if word not in word2count.keys():
            word2count[word] = 1
        else:
            word2count[word] += 1

In [92]:
for key in word2count.keys():
    word2count[key] = word2count[key]/max(word2count.values())

### Calculate the sentence score

In [94]:
sent2score = {}
for sentence in sentences:
    for word in word_tokenize(sentence):
        if word in word2count.keys():
            if len(sentence.split(' '))<30:
                if sentence not in sent2score.keys():
                    sent2score[sentence] = word2count[word]
                else:
                    sent2score[sentence] += word2count[word]

In [98]:
import heapq

best_sentences = heapq.nlargest(7, sent2score, key = sent2score.get)